In [2]:
import pandas as pd
import numpy as np
import os, sys
import cv2
import matplotlib.pyplot as plt
import copy
import tensorflow as tf
from io import StringIO # Python3 use: from io import StringIO
import seaborn as sns

models = tf.keras.models  # like 'from tensorflow.keras import models' (PyCharm import issue workaround)
layers = tf.keras.layers  # like 'from tensorflow.keras import layers' (PyCharm import issue workaround)

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

DATA = os.path.join(module_path, 'Data', 'Augmented Data')
RESULTS = os.path.join(module_path, 'Results', 'Thesis')
FIGURES = os.path.join(module_path, 'Figures', 'Thesis')
from Scripts import Data_Loader_Functions as DL
from Scripts import Model_Architectures as mA
from Scripts import Results_Evaluation as rE

In [4]:
# Define experiment names
exp_names = {
    '0-sessions-Baseline-random': 'RANDOM',
    '0-sessions-Baseline-central-pre-training': 'BCDL',
    '0-sessions-Baseline-federated-pre-training': 'BF-CNN',
    '1-sessions-Centralized-no-pre-training': 'C-CNN (N)',
    '10-sessions-Federated-central-pre-training-local-models': 'LDL',
    '11-sessions-Federated-federated-pre-training-local-models': 'FL-CNN (F)',
    '2-sessions-Centralized-pre-training': 'CDL',
    '3-sessions-Federated-no-pre-training': 'F-CNN (N)',
    '4-sessions-Federated-central-pre-training': 'FDL',
    '5-sessions-Federated-federated-pre-training': 'F-CNN (F)',
    '6-sessions-Federated-no-pre-training-personalization': 'FP-CNN (N)',
    '7-sessions-Federated-central-pre-training-personalization': 'PFDL',
    '8-sessions-Federated-federated-pre-training-personalization': 'FP-CNN (F)',
    '9-sessions-Federated-no-pre-training-local-models': 'FL-CNN (N)'
}

In [111]:
top_exp = ['BCDL', 'CDL', 'LDL', 'FDL', 'PFDL']
subjects = DL.create_pain_df(os.path.join(DATA, 'group_2'))['Person'].unique()

In [23]:
folders = [os.path.join(RESULTS, file) for file in os.listdir(RESULTS) if 'Seed' in file]
folders.sort()

In [128]:
df_concat = pd.DataFrame()
for folder in folders:
    files = [file for file in os.listdir(folder) if os.path.isfile(os.path.join(folder, file))]
    files.sort()

    for file in files:
        model_name = exp_names[file.split('_TEST')[0].split('PAIN_')[1]]
        if model_name in top_exp:
            f_path = os.path.join(folder, file)
            df = pd.read_csv(f_path)
            df['Model'] = model_name
            
            for subject in subjects:

                cols = [col for col in df.columns if 'subject_{}'.format(subject) in col or 'Model' in col]
                df_copy = df[cols].copy()
                df_copy['subject'] = subject
                df_copy.reset_index(inplace=True)
                df_copy.rename(columns={'index': 'Session'}, inplace=True)
                new_cols = [col.split('subject_{}_'.format(subject))[1] for col in cols if 'subject' in col]
                df_copy.rename(columns=dict(zip(cols, new_cols)), inplace=True)
                df_copy.rename(columns={'subject': 'Subject'}, inplace=True)
                df_copy['Seed'] = int(folder.split('Seed ')[1])
                
                df_concat = pd.concat((df_concat, df_copy))

In [130]:
df_concat = df_concat[['Seed', 'Subject', 'Model', 'Session', 'accuracy', 'auc', 'false_negatives', 'false_positives',
       'loss', 'pr', 'precision', 'recall', 'true_negatives', 'true_positives',
       ]]

In [136]:
df_concat['f1_score'] = 2 * df_concat['recall'] * df_concat['precision'] / (df_concat['recall'] + df_concat['precision'])

In [138]:
df_concat.to_csv('Detailed Results.csv')

In [145]:
df_concat[['false_negatives', 'false_positives', 'true_negatives', 'true_positives']].sum().sum() / 4 / 4

198675.0